In [180]:
import pandas as pd
import numpy as np

In [181]:
df = pd.read_csv('/Users/amitchandna/Documents/Data_Science/Github/tamu_web_scrape/texas_football/texas_boys_football_1718.csv', names=["Score", "Team_2", "Result_team_1", "Date","District","Time","Team_1","Team_1_Mascot","Address","Level","Season"])

In [182]:
s = df.Score
a = df.Address
d = df.District

In [183]:
scored = s.str.split('-', expand=True)
addressed= a.str.split(',', expand=True)
districted = d.str.split('•', expand=True)

In [184]:
scored[2].dtype

dtype('O')

In [185]:
df = pd.concat([df,scored], axis=1)

In [186]:
df = df.rename(columns={0:'Score_1', 1:'Score_2'})
df = df.drop(columns={'Score', 2})

In [187]:
df = pd.concat([df,addressed], axis=1)

In [188]:
df = df.rename(columns={0:'team_1_physical_Address',1:'team_1_City',2:'team_1_State',3:'Team_1_Zip_code'})
df = df.drop(columns={'Address',4,5})

In [189]:
df = pd.concat([df,districted], axis=1)

In [190]:
df = df.rename(columns={0:"Home_Away", 1:'District_non_district'})
df = df.drop(columns={'District'})

In [191]:
df = df[['Team_1','Score_1', 'Team_2', 'Score_2', 'Result_team_1','Team_1_Mascot','Date','District_non_district','Time','Level','Season', 'team_1_physical_Address','team_1_City','team_1_State','Team_1_Zip_code' ]]

In [192]:
df.to_csv('texas_boys_football_1718_clean.csv')

In [193]:
df.Result_team_1.value_counts()

L    6808
W    6693
Name: Result_team_1, dtype: int64

In [194]:
team_1_score = []
team_2_score = []

In [195]:
for index,row in df.iterrows():
    if row['Score_1'] == None or row['Score_2'] == None:
        row['Score_1'] = -1
        row['Score_2'] = -2
        continue
    numeric_score = ''
    for element in row['Score_1']:
        if element.isdigit():
            numeric_score += element
    if numeric_score != '':
        row['Score_1'] = int(numeric_score)
    else:
        row['Score_1'] = -3
    numeric_score = ''
    for element in row['Score_2']:
        if element.isdigit():
            numeric_score += element
    if numeric_score != '':
        row['Score_2'] = int(numeric_score)
    else:
        row['Score_2'] = -4


In [196]:
for index,row in df.iterrows():
    if row['Result_team_1'] == 'W':
        team_1_score.append(row["Score_1"] if (row["Score_1"] > row["Score_2"]) else row["Score_2"])
        team_2_score.append(row["Score_1"] if (row["Score_1"] < row["Score_2"]) else row["Score_2"])
        continue
    else:
        team_2_score.append(row["Score_1"] if (row["Score_1"] > row["Score_2"]) else row["Score_2"])
        team_1_score.append(row["Score_1"] if (row["Score_1"] < row["Score_2"]) else row["Score_2"])
        continue

In [197]:
df['Score_1'] = team_1_score
df['Score_2'] = team_2_score

In [198]:
df.to_csv('texas_football_boys_1718_clean.csv')